# PowerPoint 기반 RAG 시스템 구축

In [ ]:
%pip install pdf2image

In [ ]:
import os
from pathlib import Path

import pymupdf4llm

# PDF 파일들이 있는 폴더 경로
pdf_folder = "data/bcg_ppt"

# 폴더 내의 모든 PDF 파일 찾기
pdf_files = list(Path(pdf_folder).glob("*.pdf"))

# 각 PDF 파일을 마크다운으로 변환
md_texts = {}
for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    md_text = pymupdf4llm.to_markdown(str(pdf_file), page_chunks=True)
    md_texts[pdf_file.name] = md_text

print(f"총 {len(md_texts)}개의 PDF 파일을 처리했습니다.")

### Document 객체 생성

각 PDF 파일의 페이지별 마크다운 텍스트를 LangChain Document 객체로 변환합니다.

이를 통해 벡터 데이터베이스에서 사용할 수 있는 형태로 데이터를 준비합니다.


In [ ]:
from langchain_core.documents import Document

# md_texts의 각 PDF 파일에 대해 Document 객체들을 생성
documents = []
for pdf_name, pages in md_texts.items():
    for page in pages:
        doc = Document(
            page_content=page["text"],
            metadata={"file_name": pdf_name, **page["metadata"]},
        )
        documents.append(doc)

print(f"총 {len(documents)}개의 Document가 생성되었습니다.")

### PDF를 이미지로 변환

각 PDF 파일의 페이지를 PNG 이미지로 변환하여 저장합니다.

PyMuPDF(fitz)를 사용하여 PDF의 각 페이지를 픽스맵으로 변환한 후 이미지 파일로 저장합니다.


In [ ]:
from pathlib import Path

import pymupdf

# 각 PDF 파일을 마크다운으로 변환 및 이미지 추출
md_texts = {}
for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")

    # PDF를 이미지로 변환하여 저장
    pdf_name = pdf_file.stem  # 확장자 제외한 파일명
    output_folder = Path(pdf_folder) / f"{pdf_name}_images"
    output_folder.mkdir(exist_ok=True)

    doc = pymupdf.open(str(pdf_file))

    # 각 페이지를 이미지로 변환
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        # 픽스맵으로 변환 (기본 해상도)
        pix = page.get_pixmap()
        image_path = output_folder / f"page_{page_num + 1}.png"
        pix.save(str(image_path))
        print(f"  Saved: {image_path}")

    doc.close()

### **이미지 캡셔닝을 통한 텍스트 추출**

앞서 생성한 PDF 페이지 이미지들을 VLM 모델을 사용하여 분석하고, 각 페이지의 내용을 상세하게 텍스트로 변환합니다.

- `create_page_caption()`: PDF 페이지 이미지와 기존 추출된 텍스트를 함께 분석하여 정확한 캡션 생성
- `pdf_images_with_captions()`: 모든 PDF 이미지에 대해 캡셔닝을 수행하고 결과를 저장

In [ ]:
import base64
import time
from pathlib import Path

from dotenv import load_dotenv
from langchain_core.messages import HumanMessage

load_dotenv()
llm = ""


def create_page_caption(image_path: Path, page_text: str) -> HumanMessage | None:
    """페이지 텍스트를 컨텍스트로 활용한 캡션 생성 메시지"""
    try:
        with open(image_path, "rb") as image_file:
            encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

        prompt = f"""다음은 BCG 컨설팅 리포트의 한 페이지입니다.
        이미지를 분석하여 페이지의 내용을 정확하고 상세하게 텍스트로 변환해주세요.

참고용 추출 텍스트 (순서나 구조가 부정확할 수 있음):
{page_text[:1000]}{"..." if len(page_text) > 1000 else ""}

위 텍스트는 참고용이며, 실제 이미지의 내용과 구조가 다를 수 있습니다.
이미지를 직접 분석하여 다음과 같이 작성해주세요:

1. 페이지의 제목이나 헤더
2. 주요 텍스트 내용 (단락별로 구조화)
3. 차트, 그래프, 표가 있다면 그 내용과 데이터
4. 이미지나 다이어그램이 있다면 설명
5. 페이지 하단의 각주나 출처 정보

모든 텍스트를 읽기 쉽고 구조적으로 정리하여 한글로 작성해주세요."""

        return HumanMessage(
            content=[
                {"type": "text", "text": prompt},
                {"type": "image_url", "image_url": f"data:image/png;base64,{encoded_image}"},
            ]
        )
    except Exception as e:
        print(f"메시지 생성 오류: {e}")
        return None


def pdf_images_with_captions() -> dict[str, str]:
    """각 PDF의 페이지 이미지에 대해 캡셔닝 수행"""
    all_captions = {}

    for pdf_file in pdf_files:
        pdf_name = pdf_file.stem
        image_folder = Path(pdf_folder) / f"{pdf_name}_images"

        if not image_folder.exists():
            print(f"이미지 폴더가 없습니다: {image_folder}")
            continue

        # 해당 PDF의 documents 찾기
        pdf_documents = [doc for doc in documents if doc.metadata["file_name"] == str(pdf_file)]

        # 페이지별 이미지 파일 찾기
        image_files = sorted(image_folder.glob("page_*.png"))

        batch_data = []
        for i, image_file in enumerate(image_files):
            page_num = i  # 0부터 시작하는 페이지 번호

            # 해당 페이지의 텍스트 찾기
            page_text = ""
            if page_num < len(pdf_documents):
                page_text = pdf_documents[page_num].page_content

            batch_data.append((image_file, page_text, f"{pdf_name}_page_{page_num + 1}"))

        batch_size = 10
        for batch_idx in range(0, len(batch_data), batch_size):
            batch = batch_data[batch_idx : batch_idx + batch_size]
            batch_num = batch_idx // batch_size + 1
            total_batches = (len(batch_data) + batch_size - 1) // batch_size

            # 배치용 메시지 준비
            batch_messages = []
            batch_keys = []

            for image_file, page_text, page_key in batch:
                message = create_page_caption(image_file, page_text)
                if message:
                    batch_messages.append([message])
                    batch_keys.append(page_key)

            if batch_messages:
                try:
                    batch_results = llm.batch(batch_messages)

                    for i, result in enumerate(batch_results):
                        if i < len(batch_keys):
                            caption = result.content.strip()
                            all_captions[batch_keys[i]] = caption
                            print(f"    {batch_keys[i]}: {caption[:50]}...")

                except Exception as e:
                    print(f"  배치 처리 오류: {e}")
                    # 개별 처리로 폴백
                    for i, message_list in enumerate(batch_messages):
                        try:
                            result = llm.invoke(message_list)
                            caption = result.content.strip()
                            all_captions[batch_keys[i]] = caption
                            print(f"    {batch_keys[i]}: {caption[:50]}...")
                            time.sleep(0.1)  # 개별 요청 간 짧은 대기
                        except Exception as e:
                            print(f"    {batch_keys[i]}: 캡션 생성 실패")
                            all_captions[batch_keys[i]] = "[캡션 생성 실패]"

            if batch_num < total_batches:
                # Rate Limit
                wait_time = 1.0
                time.sleep(wait_time)

    return all_captions


# 캡셔닝 실행
captions = pdf_images_with_captions()
print(f"총 {len(captions)}개 페이지 캡션 생성됨")

# 결과 확인
for key, caption in list(captions.items())[:3]:
    print(f"\n{key}: {caption}")

In [ ]:
for doc in documents[:10]:
    # 문서 메타데이터에서 페이지 번호 추출
    page_num = doc.metadata.get("page")
    source = doc.metadata.get("file_name", "")

    # 이미지 캡션 키 생성 (pdf_images_with_captions에서 사용한 것과 동일)
    # 소스 파일명에서 확장자 제거 후 page 번호 추가
    source_basename = source.split("/")[-1].replace(".pdf", "")
    caption_key = f"{source_basename}_page_{page_num}"
    print(caption_key)

In [ ]:
# 문서 페이지 내용을 캡션으로 업데이트
# PDF 페이지에서 추출된 텍스트 대신 캡션을 사용하여 문서 객체들을 업데이트합니다.
updated_documents = []
for doc in documents:
    # 문서 메타데이터에서 페이지 번호 추출
    page_num = doc.metadata.get("page")
    source = doc.metadata.get("file_name", "")

    # 이미지 캡션 키 생성 (process_pdf_images_with_captions에서 사용한 것과 동일)
    # 소스 파일명에서 확장자 제거 후 page 번호 추가
    source_basename = source.split("/")[-1].replace(".pdf", "")
    caption_key = f"{source_basename}_page_{page_num}"

    # 해당 페이지의 캡션 가져오기
    if caption_key in captions:
        # 새로운 Document 생성 (page_content를 캡션으로 대체)
        updated_doc = Document(
            page_content=captions[caption_key],
            metadata=doc.metadata.copy(),  # 기존 메타데이터 유지
        )
        updated_documents.append(updated_doc)

print(f"\n총 {len(updated_documents)}개 문서에 대해 이미지 캡셔닝 업데이트 완료")

# 업데이트된 문서들 확인
print("\n업데이트된 문서 확인:")
for i, doc in enumerate(updated_documents[:5]):
    print(f"\n문서 {i + 1}:")
    print(f"  메타데이터: {doc.metadata}")
    print(f"  내용 (처음 100자): {doc.page_content}...")

documents = updated_documents

### 벡터 스토어 설정 및 문서 저장

Qdrant를 사용하여 하이브리드 검색(Dense + Sparse) 벡터 스토어를 설정하고, 업데이트된 문서들을 저장합니다.


In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient, models
from qdrant_client.http.models import Distance, SparseVectorParams, VectorParams

# Qdrant 클라이언트 설정
client = QdrantClient(host="localhost", port=6333)
dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 컬렉션 이름 설정
collection_name = "ppt_multimodal_rag"
try:
    client.create_collection(
        collection_name=collection_name,
        vectors_config={"dense": VectorParams(size=1536, distance=Distance.COSINE)},
    )
    print(f"새 컬렉션 '{collection_name}' 생성됨")
    qdrant = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=dense_embeddings,
        retrieval_mode=RetrievalMode.DENSE,
        vector_name="dense",
    )
    qdrant.add_documents(documents)

except Exception as e:
    print(f"에러 발생: {e}")

In [ ]:
# TODO: 이후 과정은 Agentic RAG 로 완성시켜주세요.